<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Predicting Shots Made Per Game by Kobe Bryant

_Authors: Kiefer Katovich (SF)_

---

In this lab you'll be using regularized regression penalties — ridge, lasso, and elastic net — to try and predict how many shots Kobe Bryant made per game during his career.

The Kobe Shots data set contains hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model, considering the limited number of observations (games) we have available. Plus, many of the predictors have significant multicollinearity. 


**Warning:** Some of these calculations are computationally expensive and may take a while to execute. It may be worthwhile to only use a portion of the data to perform these calculations, especially if you've experienced kernel issues in the past.

---

### 1) Load packages and data.

In [388]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [389]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2) Examine the data.

- How many columns are there?
Ans: 645
- Examine what the observations (rows) and columns represent.
Ans: rows represent individual games, columns represent features
- Why might regularization be particularly useful for modeling this data?
Ans: There might be too many unnecessary features and so regularisation helps to remove those that are not helpful in prediction and avoid overfitting.

In [390]:
# A:
kobe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1558 entries, 0 to 1557
Columns: 645 entries, SHOTS_MADE to CAREER_GAME_NUMBER
dtypes: float64(640), int64(5)
memory usage: 7.7 MB


In [391]:
kobe.columns

Index(['SHOTS_MADE', 'AWAY_GAME', 'SEASON_OPPONENT:atl:1996-97',
       'SEASON_OPPONENT:atl:1997-98', 'SEASON_OPPONENT:atl:1999-00',
       'SEASON_OPPONENT:atl:2000-01', 'SEASON_OPPONENT:atl:2001-02',
       'SEASON_OPPONENT:atl:2002-03', 'SEASON_OPPONENT:atl:2003-04',
       'SEASON_OPPONENT:atl:2004-05',
       ...
       'ACTION_TYPE:tip_layup_shot', 'ACTION_TYPE:tip_shot',
       'ACTION_TYPE:turnaround_bank_shot',
       'ACTION_TYPE:turnaround_fadeaway_bank_jump_shot',
       'ACTION_TYPE:turnaround_fadeaway_shot',
       'ACTION_TYPE:turnaround_finger_roll_shot',
       'ACTION_TYPE:turnaround_hook_shot', 'ACTION_TYPE:turnaround_jump_shot',
       'SEASON_GAME_NUMBER', 'CAREER_GAME_NUMBER'],
      dtype='object', length=645)

In [392]:
kobe

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,4.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.105263,0.0,0.000000,0.052632,62,1555
1554,4.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,63,1556
1555,9.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.045455,0.0,0.045455,0.045455,64,1557
1556,3.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.166667,0.0,0.000000,0.000000,65,1558


In [393]:
kobe.shape

(1558, 645)

In [394]:
kobe.isnull().sum().sum()

0

---

### 3) Create predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [395]:
# A:
from sklearn.preprocessing import PolynomialFeatures # to generate polynomial and interaction terms

# Create X and y.
X = kobe.drop('SHOTS_MADE', axis=1)# take all cols except, 'SHOTS_MADE' --> response
y = kobe['SHOTS_MADE']

In [396]:
# Import train_test_split.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [397]:
# Create train/test splits using X_overfit, y defined above.
# setting random_state helps with repeatability on the splits, instead of diff splits each time
# note that setting such a high test_size vs train is rarely practised, it is done so here for simulating a poor model learning here
# default test_size in sklearn=25%. rule of thumb: train_size>test_size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

In [398]:
# Scale our data to standardize different feature magnitudes.
# Relabeling scaled data as "Z" is common.(Z-score-Scaling topic in feature engineering lesson)
# note that we will only "transform()" test, applying the learning achieved by "fitting" on train 
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

In [399]:
print(f'Z_train shape is: {Z_train.shape}')
print(f'y_train shape is: {y_train.shape}')
print(f'Z_test shape is: {Z_test.shape}')
print(f'y_test shape is: {y_test.shape}')

Z_train shape is: (467, 644)
y_train shape is: (467,)
Z_test shape is: (1091, 644)
y_test shape is: (1091,)


---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?
Ans: Not very well. Mean cross validation score: 0.5674334707623419

In [400]:
# A:
# Import the appropriate library and fit our OLS model.
from sklearn.linear_model import LinearRegression

In [401]:
ols = LinearRegression() # instantiate
ols.fit(Z_train, y_train) # model fit

LinearRegression()

In [402]:
# How does the model score (R^2) on the training and test data?
print(ols.score(Z_train, y_train))
print(ols.score(Z_test, y_test))

0.8913980389697927
-3.205409190074203e+26


In [403]:
ols.coef_ # Estimated coefficients for the linear regression model

array([-4.14321615e-01, -2.54663361e+12, -4.04082834e+12, -5.26345253e+12,
        2.27629149e+13, -4.41455399e+12, -2.36707494e+13, -1.32139784e+13,
       -3.07791102e+13,  1.07812375e+13,  2.49648339e+13, -2.37626248e+12,
       -2.96759709e+12,  9.82197302e+11,  8.82698009e+10, -1.48412284e+13,
        6.39375871e+11, -1.01318808e+13, -3.78894511e+13, -3.72117341e+12,
        3.54732448e+11, -2.35246935e+13, -1.30938108e+13,  1.37748326e+13,
       -1.81514379e+13, -6.23641990e+12, -5.97361396e+12, -6.23515827e+12,
        2.92176795e+13, -4.17865306e+12,  1.19362359e+12, -1.68207709e+12,
       -2.10066316e+12,  1.38754556e+12, -9.99834175e+12, -6.62748523e+12,
        5.42871945e+12,  9.94696916e+12, -7.18719120e+13, -1.61337213e+13,
       -1.83217740e+13,  4.43106741e+13,  1.61166388e+12, -6.11194167e+12,
       -4.41455399e+12, -1.98287066e+12,  1.85433312e+13,  8.44926392e+11,
       -1.68207709e+12, -3.27034538e+13,  7.83951190e+12,  8.82698009e+10,
        7.63726684e+12,  

In [404]:
len(ols.coef_) # there are 208334! corresponding to each our features

644

In [405]:
cross_val_score(ols, Z_train, y_train, cv=10).mean()

-1.3051299426192008e+28

---

### 5) Find an optimal value for the ridge regression alpha using `RidgeCV`.

Go to the documentation and [read how RidgeCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

> *Hint: Once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that ridge performs best when searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit.


In [406]:
# A:
# Linear least squares with L2 regularization
from sklearn.linear_model import Ridge

In [407]:
# Instantiate.
# alpha is the Regularization strength. we're passing 10, instead of the default 1 for stronger regularization.
ridge_model = Ridge(alpha=10)

# Fit.
ridge_model.fit(Z_train, y_train)

# Evaluate model using R2.
print(ridge_model.score(Z_train, y_train))
print(ridge_model.score(Z_test, y_test))

0.9464844128495766
0.2784709968923821


In [408]:
# Ridge regression with "built-in" cross-validation (advancing from above approach w/o cv)
from sklearn.linear_model import RidgeCV

In [409]:
# Set up a list of ridge alphas to check.
# np.logspace generates 100 values equally between 0 and 5,
# then converts them to alphas between 10^0 and 10^5 (that is, in logscale).
r_alphas = np.logspace(0, 5, 100)

# Cross-validate over our list of ridge alphas.
# alphas: pass an Array of alpha values to try. It is still the Regularization strength
ridge_cv = RidgeCV(alphas=r_alphas, scoring='r2', cv=5).fit(Z_train, y_train)# fitting 5-fold CV

In [410]:
cross_val_score(ridge_cv, X_train, y_train, cv=5).mean()

0.6293788736644045

In [411]:
print(ridge_cv.score(Z_train, y_train))
print(ridge_cv.score(Z_test, y_test))

0.8750650011558211
0.5625904944948346


In [412]:
# getting the optimal value of alpha from ridge cv
ridge_cv.alpha_

335.1602650938841

---

### 6) Cross-validate the ridge regression $R^2$ with the optimal alpha.

Is it better than the linear regression? If so, why might this be?

In [413]:
# Instantiate.
# alpha is the Regularization strength. we're passing 10, instead of the default 1 for stronger regularization.
ridge_model_optimal = Ridge(alpha=ridge_cv.alpha_)

# Fit.
ridge_model_optimal.fit(Z_train, y_train)

# Evaluate model using R2.
print(ridge_model_optimal.score(Z_train, y_train))
print(ridge_model_optimal.score(Z_test, y_test))

0.8750650011558211
0.5625904944948346


In [414]:
ridge_optimal = Ridge(alpha=ridge_cv.alpha_).fit(Z_train, y_train)# fitting using optimal alpha
cross_val_score(ridge_optimal, X_train, y_train, cv=5).mean()

0.6326992686370898

---

### 7) Find an optimal value for lasso regression alpha using `LassoCV`.

Go to the documentation and [read how LassoCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). It is very similar to `RidgeCV`.

> *Hint: Again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that lasso, unlike ridge, performs best when searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide what alphas to use itself by setting the keyword argument `n_alphas=` to however many alphas you want it to search over. We recommend letting scikit-learn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if it's working, set `verbose =1`._

In [415]:
# A:
# Imports similar to Ridge, this time for Lasso instead
from sklearn.linear_model import Lasso, LassoCV

In [416]:
# Reminder of results from evaluations before this
print(" OLS ".center(18, "="))# syntax: str.center(width, fillchar=' ')
print(ols.score(Z_train, y_train))
print(ols.score(Z_test, y_test))
print()
print(" Ridge ".center(18, "="))
print(ridge_cv.score(Z_train, y_train))
print(ridge_cv.score(Z_test, y_test))

====== OLS =======
0.8913980389697927
-3.205409190074203e+26

===== Ridge ======
0.8750650011558211
0.5625904944948346


In [417]:
# Set up a list of Lasso alphas to check.
# np.logspace generates 100 values equally between -3 and 0,
# then converts them to alphas between 10^-3 and 1 (that is, in logscale).
l_alphas = np.logspace(-3, 0, 100)

# Cross-validate over our list of Lasso alphas.
lasso_cv = LassoCV(alphas=l_alphas, cv=5, max_iter=50000).fit(Z_train, y_train);

In [418]:
print(lasso_cv.score(Z_train, y_train))
print(lasso_cv.score(Z_test, y_test))

0.7196668311537471
0.6149011264405455


In [419]:
# getting the optimal value of alpha from ridge cv
lasso_cv.alpha_

0.16297508346206452

---

### 8) Cross-validate the lasso $R^2$ with the optimal alpha.

Is it better than the linear regression? Is it better than ridge? What do the differences in results imply about the issues with the data set?

In [420]:
# A:
lasso = Lasso(alpha=lasso_cv.alpha_, max_iter=50000).fit(Z_train, y_train);

In [421]:
cross_val_score(lasso, X_train, y_train, cv=5).mean()

0.633037042098754

---

### 9) Look at the coefficients for variables in the lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original data set are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** If you only fit the lasso within `cross_val_score`, you'll have to refit it outside of that function to pull out the coefficients.

In [422]:
# A:
sorted(np.absolute(lasso_cv.coef_),reverse=True)

[0.8515303883972873,
 0.7865518923408376,
 0.6210117586468541,
 0.4242843054534741,
 0.42378982331443155,
 0.16493637533469765,
 0.15486246131194842,
 0.14984757547359573,
 0.13490928116085993,
 0.11330833135914985,
 0.10299717076025705,
 0.0810146293830309,
 0.08085938837752574,
 0.0744124741360372,
 0.07189895540854414,
 0.06457132505889819,
 0.06435173416666012,
 0.05994883288904505,
 0.05966384180662969,
 0.05779618541171727,
 0.05420645568024566,
 0.05188797614293573,
 0.05088263442755945,
 0.05060056257540534,
 0.049635151379158575,
 0.04410094141115316,
 0.04182929246470884,
 0.03824086118668168,
 0.038193673061576165,
 0.028322156173468806,
 0.02735303694966392,
 0.020819622638228032,
 0.02064626581765416,
 0.018979969436110244,
 0.017637459241926765,
 0.01711760891586881,
 0.01634568315907754,
 0.01633944560392183,
 0.01302978865815941,
 0.012150445740555675,
 0.011894706057314502,
 0.01130212227388311,
 0.007243148186839978,
 0.006581326904682039,
 0.006459416606706969,
 0.00

In [423]:
print(f'{(1-np.count_nonzero(lasso_cv.coef_)/len(lasso_cv.coef_))*100}% of variables are "zeroed-out"')

92.70186335403727% of variables are "zeroed-out"


In [424]:
X.columns[np.where(lasso_cv.coef_==lasso_cv.coef_[lasso_cv.coef_==max(lasso_cv.coef_)])]

Index(['SHOT_TYPE:2pt_field_goal'], dtype='object')

In [425]:
X.columns[np.nonzero(lasso_cv.coef_)]

Index(['SEASON_OPPONENT:bos:2001-02', 'SEASON_OPPONENT:bos:2008-09',
       'SEASON_OPPONENT:dal:2003-04', 'SEASON_OPPONENT:dal:2012-13',
       'SEASON_OPPONENT:den:2006-07', 'SEASON_OPPONENT:det:2011-12',
       'SEASON_OPPONENT:det:2015-16', 'SEASON_OPPONENT:gsw:2000-01',
       'SEASON_OPPONENT:gsw:2010-11', 'SEASON_OPPONENT:ind:2002-03',
       'SEASON_OPPONENT:lac:2003-04', 'SEASON_OPPONENT:mem:2009-10',
       'SEASON_OPPONENT:mia:2012-13', 'SEASON_OPPONENT:mil:2009-10',
       'SEASON_OPPONENT:njn:2000-01', 'SEASON_OPPONENT:njn:2006-07',
       'SEASON_OPPONENT:noh:2011-12', 'SEASON_OPPONENT:nyk:2008-09',
       'SEASON_OPPONENT:okc:2014-15', 'SEASON_OPPONENT:orl:2000-01',
       'SEASON_OPPONENT:phx:2007-08', 'SEASON_OPPONENT:por:2003-04',
       'SEASON_OPPONENT:por:2005-06', 'SEASON_OPPONENT:sac:1999-00',
       'SEASON_OPPONENT:sas:2014-15', 'SEASON_OPPONENT:sea:1998-99',
       'SEASON_OPPONENT:tor:2004-05', 'SEASON_OPPONENT:uta:2001-02',
       'SEASON_OPPONENT:uta:2015-1

---

### 10) Find an optimal value for elastic net regression alpha using `ElasticNetCV`.

Go to the documentation and [read how ElasticNetCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note that here you'll be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: Strength of regularization.
- `l1_ratio`: Amount of ridge vs. lasso (0 = all ridge, 1 = all lasso).
    
Do not include 0 in the search for `l1_ratio` — it won't allow it and will break.

You can use `n_alphas` for the alpha parameters instead of setting your own values, which we highly recommend.

Also, be careful setting too many l1_ratios over cross-validation folds in your search. It can take a long time if you choose too many combinations and, for the most part, there are diminishing returns in this data.

In [426]:
# A:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

In [427]:
# Instantiate model.
enet_model = ElasticNetCV(cv=5)

# Fit model using optimal alpha.
enet_model = enet_model.fit(X_train, y_train)

# Generate predictions.
enet_model_preds = enet_model.predict(X_test)
enet_model_preds_train = enet_model.predict(X_train)

# Evaluate model.
print(enet_model.score(X_train, y_train))
print(enet_model.score(X_test, y_test))

0.6270896676579155
0.6016530116245435


In [428]:
# Here is the optimal value of alpha.
enet_model.alpha_

1.2679102751628921

In [429]:
enet_model.l1_ratio_

0.5

In [430]:
enet_model = ElasticNet(alpha=enet_model.alpha_,l1_ratio=enet_model.l1_ratio_)

---

### 11) Cross-validate the elastic net $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the ridge and lasso regularized regressions?
Ans: It performed worse than both Ridge and Lasso regularized regressions.

In [433]:
# A:
cross_val_score(enet_model, X_train, y_train, cv=5).mean()

0.6027926243442903

---

### 12. [Bonus] Compare the residuals for ridge and lasso visually.


In [ ]:
# A: Maybe a jointplot?